In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Funzione per calcolare la distanza euclidea

In [138]:
def euclideanDistance(ex, query):
    distance = np.linalg.norm(ex - query)
    return distance

Visto che peseremo i risultati in base alla distanza serve una funzione per definire il peso

In [139]:
def weightByDistance(distance):
    weight = 1/distance
    return weight

K-NN pesato

In [140]:
'''def kNN_weighted(examples, query, k):
    class_distance = []
    for i in range(examples.shape[0]):
        classe = examples.at[i, 'hazardous']
        single_example = examples.loc[i].drop('hazardous').values
        distance = euclideanDistance(single_example, query.values)
        if distance == 0:
            return classe
        class_distance.append((distance, classe))

    class_distance = sorted(class_distance)[:k]
    freq_hazardous = 0
    freq_not_hazardous = 0

    for example in class_distance:
        if example[1] == 'True':
            freq_hazardous = freq_hazardous + weightByDistance(example[0])
        else:
            freq_not_hazardous = freq_not_hazardous + weightByDistance(example[0])

    if freq_hazardous  > freq_not_hazardous :
        return 'True'
    else:
        return 'False'
'''

def kNN_weighted(examples, query, k):
    class_distance = []
    query_values = query.values
    query_weight = weightByDistance(np.linalg.norm(examples.drop('hazardous', axis=1) - query_values, axis=1))
    class_distance = list(zip(query_weight, examples['hazardous']))

    class_distance.sort(reverse=True)  # Ordinamento basato sui pesi

    freq_hazardous = sum(w for w, c in class_distance[:k] if c == True)
    freq_not_hazardous = sum(w for w, c in class_distance[:k] if c == False)

    if freq_hazardous > freq_not_hazardous:
        return True
    else:
        return False



Matrice di confusione

In [141]:
def confusionMatrix(classi):
    # True hazardous
    TH = classi.loc[(classi['hazardous'] == True) & (classi['hazardous_predicted'] == True)].reset_index(drop=True).shape[0] 
    # False hazardous
    FH = classi.loc[(classi['hazardous'] == False) & (classi['hazardous_predicted'] == True)].reset_index(drop=True).shape[0]
    # False not hazardous
    FNH = classi.loc[(classi['hazardous'] == True) & (classi['hazardous_predicted'] == False)].reset_index(drop=True).shape[0]
    # True not hazardous
    TNH = classi.loc[(classi['hazardous'] == False) & (classi['hazardous_predicted'] == False)].reset_index(drop=True).shape[0]
    print(TH, FH, FNH, TNH)
    conf_matrix = pd.DataFrame({"True": [TH, FH], "False": [FNH, TNH]}, index = ["True", "False"])
    
    return(conf_matrix, TH, FH, FNH, TNH)

Valutazione

In [142]:
def evaluation(TH, FH, FNH, TNH):
    m = TH + FH + FNH + TNH

    # classificazioni corrette
    correct_classifications = TH + TNH
    accuracy = correct_classifications / m
    accuracy_percentage = accuracy * 100
    print("Accuratezza del modello (istanze classificate correttamente) => " + str(round(accuracy_percentage, 5)) + "\nN° classificazioni corrette =>" + str(correct_classifications))

    #classificazioni errate
    incorrect_classifications = FH + FNH
    inaccuracy = incorrect_classifications / m
    inaccuracy_percentage = inaccuracy * 100
    print("Imprecisione del modello (istanze classificate incorrettamente) => " + str(round(inaccuracy_percentage, 5)) + "\nN° classificazioni sbagliate =>" + str(incorrect_classifications))

    #coeff Kappa
    p_true = ((TH + FNH)/m) * ((TNH + FH)/m)
    p_false = ((TNH + FH)/m) * ((TNH + FNH)/m)
    p_e = p_true + p_false
    k_coeff = (accuracy - p_e) / (1 - p_e)
    print("Kappa coefficient => " + str(round(k_coeff, 5)))

    print("\n######## STATISTICHE DELLE DUE CLASSI ########")
    print("\n++++++++ HAZARDOUS ++++++++")
    precision = TH / (TH + FH)
    print("Precisione = " + str(round(precision, 5)))
    recall = TH / (TH + FNH)
    print("Recall = " + str(round(recall, 5)))
    F1_score = 2 * ((precision * recall) / (precision + recall))
    print("F1 score = " + str(round(F1_score, 5)))

    print("\n-------- NOT HAZARDOUS -------")
    precision = TNH / (TNH + FNH)
    print("Precisione = " + str(round(precision, 5)))
    recall = TNH / (TNH + FH)
    print("Recall = " + str(round(recall, 5)))
    F1_score = 2 * ((precision * recall) / (precision + recall))
    print("F1 score = " + str(round(F1_score, 5)))


caricamento del dataset e definizione del numero di vicini

In [143]:
dataset = pd.read_csv('neo_v2_edited.csv')

dataset = dataset.sample(frac=1).reset_index(drop=True)

k=20

In [144]:
print('\n******* TEST MODE: SPLIT 70% TRAIN, REMAINDER TEST *******')

# Splitting training/test set (70% training set e 30% test set)
training_set = dataset.head(round(len(dataset)*(70/100))) # arrotondo per eccesso
test_set = dataset.tail(len(dataset) - len(training_set)).reset_index(drop=True)
test_set['hazardous_predicted'] = np.zeros(test_set.shape[0])    # viene inserita la classe predetta

for i in range(test_set.shape[0]):
    if i%100 == 0:
        print('Predico query numero ' + str(i))
    query = test_set[i:(i+1)].drop('hazardous', axis = 1).drop('hazardous_predicted', axis = 1).reset_index(drop=True)
    test_set.at[i,'hazardous_predicted'] = kNN_weighted(training_set, query, k)


******* TEST MODE: SPLIT 70% TRAIN, REMAINDER TEST *******
Predico query numero 0
Predico query numero 100
Predico query numero 200
Predico query numero 300
Predico query numero 400
Predico query numero 500
Predico query numero 600
Predico query numero 700
Predico query numero 800
Predico query numero 900
Predico query numero 1000


C:\Users\ricca\AppData\Local\Temp\ipykernel_804\1270757608.py:2: RuntimeWarning: divide by zero encountered in divide
  weight = 1/distance


Predico query numero 1100
Predico query numero 1200
Predico query numero 1300
Predico query numero 1400
Predico query numero 1500
Predico query numero 1600
Predico query numero 1700
Predico query numero 1800
Predico query numero 1900
Predico query numero 2000
Predico query numero 2100
Predico query numero 2200
Predico query numero 2300
Predico query numero 2400
Predico query numero 2500
Predico query numero 2600
Predico query numero 2700
Predico query numero 2800
Predico query numero 2900
Predico query numero 3000
Predico query numero 3100
Predico query numero 3200
Predico query numero 3300
Predico query numero 3400
Predico query numero 3500
Predico query numero 3600
Predico query numero 3700
Predico query numero 3800
Predico query numero 3900
Predico query numero 4000
Predico query numero 4100
Predico query numero 4200
Predico query numero 4300
Predico query numero 4400
Predico query numero 4500
Predico query numero 4600
Predico query numero 4700
Predico query numero 4800
Predico quer

Valutazione

In [145]:
print('\n------- MATRICE DI CONFUSIONE -------')
classi = test_set[['hazardous','hazardous_predicted']]
(confusione, TH, FH, FNH, TNH) = confusionMatrix(classi)
print(confusione)

evaluation(TH, FH, FNH, TNH)


------- MATRICE DI CONFUSIONE -------
17 25 636 7549
       True  False
True     17    636
False    25   7549
Accuratezza del modello (istanze classificate correttamente) => 91.96548
N° classificazioni corrette =>7566
Imprecisione del modello (istanze classificate incorrettamente) => 8.03452
N° classificazioni sbagliate =>661
Kappa coefficient => -6.30413

######## STATISTICHE DELLE DUE CLASSI ########

++++++++ HAZARDOUS ++++++++
Precisione = 0.40476
Recall = 0.02603
F1 score = 0.04892

-------- NOT HAZARDOUS -------
Precisione = 0.9223
Recall = 0.9967
F1 score = 0.95806
